<a href="https://colab.research.google.com/github/amaar-dev/Face_mask_DT/blob/main/Face_mask_DT_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training model to recognise face masks

## Importing Data set to train Model

In [3]:
from pathlib import Path
import kagglehub
import os


path = kagglehub.dataset_download("vijaykumar1799/face-mask-detection")

print("Path to dataset files:", path)

dataset_dir = Path(path) / "Dataset"
with_mask_dir = dataset_dir / "with_mask"
without_mask_dir = dataset_dir / "without_mask"


#check folder
print("Files in dataset folder: ")
print(os.listdir(path))


print("with_mask : ", len(os.listdir(with_mask_dir)), "images")
print("without_mask : ", len(os.listdir(without_mask_dir)), "images")





Path to dataset files: /kaggle/input/face-mask-detection
Files in dataset folder: 
['Dataset']
with_mask :  2994 images
without_mask :  2994 images
